<a href="https://colab.research.google.com/github/NJR203Group2/palace_museum/blob/main/palacemuseum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request as req
import bs4 as bs
import json
import pandas as pd
from urllib.parse import urljoin
import os
import requests

table = []
url = "https://www.npm.gov.tw/Calendar.aspx?sno=02012623&type=1"
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)

base = "https://www.npm.gov.tw/"
res = requests.get(url)
res.encoding = "utf-8"
soup = bs.BeautifulSoup(res.text, "html.parser")
# 找出所有展覽卡片
os.makedirs("images", exist_ok=True)

rs = html.find_all("a",{"class":"card"})
for r in rs:
  # 展覽名稱 連結 時間 種類 地點 展室
  en = r["href"]
  en_href = base + en
  name = r.find("div",{"class":"card-content-top"})
  title = name.find("h3",{"class":"card-title h5"}) # div內含h3

  date_div = title.find_next_sibling("div") # h3同階的div
  sort = r.find("div",{"class":"card-tags"})
  area = r.find("div",{"class":"card-content-bottom"})
  location = area.find("div").contents[0]
  room = area.find("span")

  title_text = title.get_text().strip()
  time_text = date_div.get_text().strip()
  sort_text = sort.get_text().strip()
  location_text = location.get_text().strip()
  room_text = room.get_text().strip()

  # print(name_text)
  # print(time_text)
  # print(sort_text)
  # print(location_text)
  # print(room_text)

  # 圖片
  img_tag = r.select_one("img")
  src = img_tag.get("data-src") or img_tag.get("src") if img_tag else None
  if not src or src.lower().endswith(".gif"):
      continue
  full_img_url = urljoin(base, src)

  row={
    "展覽名稱":title_text,
    "連結":en_href,
    "時間":time_text,
    "種類":sort_text,
    "地點":location_text,
    "展室":room_text,
    "圖片連結":full_img_url
  }
  table.append(row)

if len(table) == 0:
  print("未找到任何展覽資料")
  print("程式結束")
  exit()

df = pd.DataFrame(table)
df['種類'] = df['種類'].str.replace("#","", regex=False)
# 篩選地點 "北部院區　第一展覽館"
target = "北部院區　第一展覽館"
fil = df["地點"]==target
dffil = df[fil]
#dffil
# 輸出json格式
#dffil = dffil.reset_index(drop=True)
#dffil.to_json("palacemuseum.json", orient="index", force_ascii=False, indent=4)
# 輸出csv格式
dffil.to_csv("palacemuseum.csv",encoding="utf-8",index=False)